## IMPORTANT, THIS CODE REUSES THE CODE FROM THE 2 PREVIOUS JUPYTER NOTEBOOKS
it basically does the same as we did before with all the stocks, but for each industry subset of stocks (GICS Sector) and make an the industry weighted portfolio at the end
but it is not commented as much (at all), to understand it, read the previous notebooks

# Imports

In [1]:
import pandas as pd 
import numpy as np
import os
from pathlib import Path
from collections import Counter
from datetime import datetime, timedelta
import seaborn as sns

dataPath = str(Path(os.getcwd()).parent.absolute())+"/data"

# Reading the data

In [2]:
allStocksData = pd.read_csv(dataPath+'/raw/stocksData.csv', index_col=0) 
totalMarketData = pd.read_csv(dataPath+'/interim/marketData.csv', index_col=0)
sectors = pd.read_excel(dataPath+'/raw/sectors.xlsx', index_col=0) 

In [7]:
industryWeightedGreenPortfolios = pd.DataFrame(index=allStocksData.index, data=0.0, columns=["MKT_GREEN_BL|DAILY_RETURNS","MKT_BROWN_BL|DAILY_RETURNS","MKT_GMB_BL|DAILY_RETURNS","MKT_GREEN_RO|DAILY_RETURNS","MKT_BROWN_RO|DAILY_RETURNS","MKT_GMB_RO|DAILY_RETURNS"])

for sector in sorted(set(sectors["GICS Sector"])):


    stocksData = pd.DataFrame(index=allStocksData.index)
    for column in allStocksData:
        if sectors["GICS Sector"][column[0: column.find("|") ]] == sector :
            stocksData[column]=allStocksData[column]

    stocksData.to_csv(dataPath+"/interim/industries/industry"+str(sector)+"StocksData.csv")
    
    
    ######################################################
    # Making the industry portfolio
    ######################################################
    
    # Create the df
    marketData = pd.DataFrame(index=stocksData.index, columns=["MKT|DAILY_RETURNS","MKT|MKT_CAP"])

    #create a list of all the stocks market cap column names
    stocksMktCapColumns=[]
    for column in stocksData:
        if column[-8:] == "|MKT_CAP":
            stocksMktCapColumns+=[column]

    #sum all stocks market cap to have our market market cap
    marketData['MKT|MKT_CAP'] = list(stocksData[stocksMktCapColumns].sum(axis=1))

    #creating the daily returns with the market cap changes
    lastDayMktCap=marketData["MKT|MKT_CAP"]["2016-09-13"] #for day one it doesn't work so we set it to 0% (we'll delete the first row later)
    for index, row in marketData.iterrows():
        marketData["MKT|DAILY_RETURNS"][index]=100*((marketData["MKT|MKT_CAP"][index]/lastDayMktCap)-1)
        lastDayMktCap=marketData["MKT|MKT_CAP"][index]

    marketData.to_csv(dataPath+"/interim/industries/industry"+str(sector)+"MarketData.csv")
    
    #################################################################################
    # Ranking and normalising ESG Scores And creating a Green minus Brown portfolio
    #################################################################################
    
    stocksNormalizedData = stocksData.copy()
    
    #deleteting the value and changing the type from int to float for the esg ranking
    #takes a lot of time, should be optimized, not very smart to do it for each industry...
    
    for column in stocksData:
            if column[-22:] == "|ESG_RANKING_ROBECOSAM":
                stocksNormalizedData[column]=0.0
                
    #Robecosam ratings
    
    #Not optimized very slow
    for index, row in stocksData.iterrows():
        dailyESGRankings=[]
        for column in stocksData:
            if column[-22:] == "|ESG_RANKING_ROBECOSAM":
                dailyESGRankings+=[stocksData[column][index]]
        dailyESGRankings.sort(reverse=True)

        worstScoreRank=-1
        if len(dailyESGRankings)-1!= dailyESGRankings.index(dailyESGRankings[-1]):
            worstScoreRank=dailyESGRankings.index(dailyESGRankings[-1])

        for column in stocksData:
            if column[-22:] == "|ESG_RANKING_ROBECOSAM":

                if dailyESGRankings.index(stocksData[column][index]) == worstScoreRank:
                    stocksNormalizedData[column][index]=1
                else:
                    stocksNormalizedData[column][index] = dailyESGRankings.index(stocksData[column][index])/len(dailyESGRankings)

    
    #Now for Bloomberg
    
    #Not optimized very slow
    for index, row in stocksData.iterrows():
        dailyESGRankings=[]

        for column in stocksData:
            if column[-20:] == "|ESG_SCORE_BLOOMBERG":
                dailyESGRankings+=[stocksData[column][index]]
        dailyESGRankings.sort(reverse=True)


        worstScoreRank=-1
        if len(dailyESGRankings)-1!= dailyESGRankings.index(dailyESGRankings[-1]):
            worstScoreRank=dailyESGRankings.index(dailyESGRankings[-1])

        for column in stocksData:
            if column[-20:] == "|ESG_SCORE_BLOOMBERG":

                if dailyESGRankings.index(stocksData[column][index]) == worstScoreRank:
                    stocksNormalizedData[column][index]=1
                else:
                    stocksNormalizedData[column][index] = dailyESGRankings.index(stocksData[column][index])/len(dailyESGRankings) 

    
    marketGreenPortfolios = pd.DataFrame(index=stocksData.index, columns=["MKT_GREEN_BL|DAILY_RETURNS","MKT_BROWN_BL|DAILY_RETURNS","MKT_GMB_BL|DAILY_RETURNS","MKT_GREEN_RO|DAILY_RETURNS","MKT_BROWN_RO|DAILY_RETURNS","MKT_GMB_RO|DAILY_RETURNS"])

    lastDay=stocksData.index[0]
    for index, row in stocksData[stocksData.index > stocksData.index[0]].iterrows(): #skip the fisrt day
        countBloombergGreenStocks, countBloombergBrownStocks, countRobecosamGreenStocks, countRobecosamBrownStocks = 0, 0, 0 ,0
        returnsBloombergGreenStocks, returnsBloombergBrownStocks, returnsRobecosamGreenStocks, returnsRobecosamBrownStocks = 0, 0, 0, 0

        for column in stocksNormalizedData:
            if column[-20:] == "|ESG_SCORE_BLOOMBERG":
                if stocksNormalizedData[column][lastDay]<0.3:
                    countBloombergGreenStocks+=1

                    returnsBloombergGreenStocks+=stocksNormalizedData[column[:-20]+"|DAILY_RETURNS"][index]
                elif stocksNormalizedData[column][lastDay]>0.7:
                    countBloombergBrownStocks+=1
                    returnsBloombergBrownStocks+=stocksNormalizedData[column[:-20]+"|DAILY_RETURNS"][index]
            elif column[-22:] == "|ESG_RANKING_ROBECOSAM":
                if stocksNormalizedData[column][lastDay]<0.3:
                    countRobecosamGreenStocks+=1
                    returnsRobecosamGreenStocks+=stocksNormalizedData[column[:-22]+"|DAILY_RETURNS"][index]
                elif stocksNormalizedData[column][lastDay]>0.7:
                    countRobecosamBrownStocks+=1
                    returnsRobecosamBrownStocks+=stocksNormalizedData[column[:-22]+"|DAILY_RETURNS"][index]

        marketGreenPortfolios["MKT_GREEN_BL|DAILY_RETURNS"][index]=returnsBloombergGreenStocks/countBloombergGreenStocks
        marketGreenPortfolios["MKT_BROWN_BL|DAILY_RETURNS"][index]=returnsBloombergBrownStocks/countBloombergBrownStocks
        marketGreenPortfolios["MKT_GREEN_RO|DAILY_RETURNS"][index]=returnsRobecosamGreenStocks/countRobecosamGreenStocks
        marketGreenPortfolios["MKT_BROWN_RO|DAILY_RETURNS"][index]=returnsRobecosamBrownStocks/countRobecosamBrownStocks

    marketGreenPortfolios["MKT_GMB_BL|DAILY_RETURNS"]=marketGreenPortfolios["MKT_GREEN_BL|DAILY_RETURNS"]- marketGreenPortfolios["MKT_BROWN_BL|DAILY_RETURNS"]
    marketGreenPortfolios["MKT_GMB_RO|DAILY_RETURNS"]=marketGreenPortfolios["MKT_GREEN_RO|DAILY_RETURNS"]- marketGreenPortfolios["MKT_BROWN_RO|DAILY_RETURNS"]

    marketGreenPortfolios["INDUSTRY_WEIGHT"]=marketData["MKT|MKT_CAP"]/totalMarketData["MKT|MKT_CAP"]
    for column in industryWeightedGreenPortfolios:
        industryWeightedGreenPortfolios[column]+=marketGreenPortfolios[column]*marketGreenPortfolios["INDUSTRY_WEIGHT"]

    marketGreenPortfolios.to_csv(dataPath+"/interim/industries/industry"+str(sector)+"GreenPortfolios.csv")
    

industryWeightedGreenPortfolios.to_csv(dataPath+"/interim/industryWeightedGreenPortfolios.csv")
    




/tmp/ipykernel_166965/529774694.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  marketData["MKT|DAILY_RETURNS"][index]=100*((marketData["MKT|MKT_CAP"][index]/lastDayMktCap)-1)
/tmp/ipykernel_166965/529774694.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  marketData["MKT|DAILY_RETURNS"][index]=100*((marketData["MKT|MKT_CAP"][index]/lastDayMktCap)-1)
/tmp/ipykernel_166965/529774694.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a 

In [21]:
industryWeightedGreenPortfolios["MKT_BROWN|DAILY_RETURNS"]=industryWeightedGreenPortfolios["MKT_BROWN_BL|DAILY_RETURNS"]/2+industryWeightedGreenPortfolios["MKT_BROWN_RO|DAILY_RETURNS"]/2
industryWeightedGreenPortfolios["MKT_GREEN|DAILY_RETURNS"]=industryWeightedGreenPortfolios["MKT_GREEN_BL|DAILY_RETURNS"]/2+industryWeightedGreenPortfolios["MKT_GREEN_RO|DAILY_RETURNS"]/2
industryWeightedGreenPortfolios["MKT_GMB|DAILY_RETURNS"]=industryWeightedGreenPortfolios["MKT_GMB_BL|DAILY_RETURNS"]/2+industryWeightedGreenPortfolios["MKT_GMB_RO|DAILY_RETURNS"]/2

industryWeightedGreenPortfolios.columns = industryWeightedGreenPortfolios.columns.str.removeprefix('MKT')
industryWeightedGreenPortfolios = industryWeightedGreenPortfolios.add_prefix('IND_W')

industryWeightedGreenPortfolios.to_csv(dataPath+"/interim/industryWeightedGreenPortfolios.csv")

KeyError: 'MKT_BROWN_BL|DAILY_RETURNS'

In [17]:
industryWeightedGreenPortfolios3 = industryWeightedGreenPortfolios.copy()

In [20]:
industryWeightedGreenPortfolios

,IND_W_GREEN_BL|DAILY_RETURNS,IND_W_BROWN_BL|DAILY_RETURNS,IND_W_GMB_BL|DAILY_RETURNS,IND_W_GREEN_RO|DAILY_RETURNS,IND_W_BROWN_RO|DAILY_RETURNS,IND_W_GMB_RO|DAILY_RETURNS,IND_W_BROWN|DAILY_RETURNS,IND_W_GREEN|DAILY_RETURNS,IND_W_GMB|DAILY_RETURNS
Dates,,,,,,,,,
2016-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-09-14,-0.316179,-0.094972,-0.221207,-0.222855,-0.06236,-0.160495,-0.078666,-0.269517,-0.190851
2016-09-15,1.004141,1.348165,-0.344024,0.956266,1.134588,-0.178322,1.241376,0.980204,-0.261173
2016-09-16,-0.426916,-0.361097,-0.065818,-0.360297,-0.259571,-0.100725,-0.310334,-0.393606,-0.083272
2016-09-19,0.155257,0.217109,-0.061852,0.095385,0.072793,0.022592,0.144951,0.125321,-0.01963
...,...,...,...,...,...,...,...,...,...
2022-11-29,0.304463,0.050648,0.253815,0.166389,0.081256,0.085133,0.065952,0.235426,0.169474
2022-11-30,2.805504,2.530667,0.274836,3.016337,2.781652,0.234685,2.65616,2.91092,0.254761
2022-12-01,-0.204554,0.193338,-0.397892,-0.112762,0.029037,-0.1418,0.111188,-0.158658,-0.269846
